In [ ]:
from collections import namedtuple

import numpy as np

import sympy as sym
from sympy import Symbol, symbols, Matrix, ImmutableMatrix, MatrixSymbol, Function, simplify, diff
from sympy.utilities.codegen import codegen
from sympy.utilities.lambdify import implemented_function
from sympy.utilities.iterables import numbered_symbols
from sympy.codegen.ast import Assignment, FunctionDefinition
from sympy.printing.cxx import CXX11CodePrinter
from sympy.codegen.rewriting import create_expand_pow_optimization

In [ ]:
NamedExpr = namedtuple("NamedExpr", ["name", "expr"])

def name_expr(name, expr):
    if hasattr(expr, "shape"):
        s = sym.MatrixSymbol(name, *expr.shape)
    else:
        s = Symbol(name)
    return NamedExpr(s, expr)

In [ ]:
# lambda for q/p
l = Symbol("lambda")

# h for step length
h = Symbol("h")

# p for position
p = MatrixSymbol("p", 3, 1)

# d for direction
d = MatrixSymbol("d", 3, 1)

# time
t = Symbol("t")

# mass
m = Symbol("m")

# absolute momentum
p_abs = Symbol("p_abs")

In [ ]:
B1 = MatrixSymbol("B1", 3, 1)
B2 = MatrixSymbol("B2", 3, 1)
B3 = MatrixSymbol("B3", 3, 1)
k1 = name_expr("k1", l * d.as_explicit().cross(B1))
p2 = name_expr("p2", p + h/2 * d + h**2/8 * k1.name)

k2 = name_expr("k2", l * (d + h/2 * k1.name).as_explicit().cross(B2))
k3 = name_expr("k3", l * (d + h/2 * k2.name).as_explicit().cross(B2))
p3 = name_expr("p3", p + h * d + h**2/2 * k3.name)

k4 = name_expr("k4", l * (d + h * k3.name).as_explicit().cross(B3))

err = name_expr("err", h**2 * (k1.name - k2.name - k3.name + k4.name).as_explicit().norm(1))

new_p = name_expr("new_p", p + h * d + h**2/6 * (k1.name + k2.name + k3.name))
new_d = name_expr("new_d", d + h/6 * (k1.name + 2 * (k2.name + k3.name) + k4.name))
new_d_norm = name_expr("new_d", new_d.expr / new_d.expr.as_explicit().norm())

dtds = name_expr("dtds", sym.sqrt(1 + m**2 / p_abs**2))
new_time = name_expr("new_time", h * dtds.name)

In [ ]:
# get the full jacobian by substituting all the expressions above

In [ ]:
dk1dTL = name_expr("dk1dTL", k1.expr.jacobian([d, l]))
dk2dTL = name_expr("dk2dTL", k2.expr.jacobian([d, l]) + k2.expr.jacobian(k1.name) * dk1dTL.expr)
dk3dTL = name_expr("dk3dTL", k3.expr.jacobian([d, l]) + k3.expr.jacobian(k2.name) * dk2dTL.name.as_explicit())
dk4dTL = name_expr("dk4dTL", k4.expr.jacobian([d, l]) + k4.expr.jacobian(k3.name) * dk3dTL.name.as_explicit())

dFdTL = name_expr("dFdTL",
    new_p.expr.as_explicit().jacobian([p, l]) +
    new_p.expr.as_explicit().jacobian(k1.name) * dk1dTL.expr +
    new_p.expr.as_explicit().jacobian(k2.name) * dk2dTL.name.as_explicit() +
    new_p.expr.as_explicit().jacobian(k3.name) * dk3dTL.name.as_explicit())
dGdTL = name_expr("dGdTL",
    new_d.expr.as_explicit().jacobian([p, l]) +
    new_d.expr.as_explicit().jacobian(k1.name) * dk1dTL.expr +
    new_d.expr.as_explicit().jacobian(k2.name) * dk2dTL.name.as_explicit() +
    new_d.expr.as_explicit().jacobian(k3.name) * dk3dTL.name.as_explicit() +
    new_d.expr.as_explicit().jacobian(k4.name) * dk4dTL.name.as_explicit())

D = sym.eye(8)
D[0:3,4:8] = dFdTL.expr
D[4:7,4:8] = dGdTL.expr
D[3,7] = h * m**2 * l / dtds.name

J = Matrix(MatrixSymbol("J", 8, 8).as_explicit())
for indices in np.ndindex(J.shape):
    if D[indices] in [0, 1]:
        J[indices] = D[indices]
J = ImmutableMatrix(J)
new_J = name_expr("new_J", J * D)

In [ ]:
# get the full jacobian step by step with intermediate expressions

In [ ]:
B1 = MatrixSymbol("B1", 3, 1)
B2 = MatrixSymbol("B2", 3, 1)
B3 = MatrixSymbol("B3", 3, 1)
k1 = name_expr("k1", l * d.as_explicit().cross(B1))
p2 = name_expr("p2", p + h/2 * d + h**2/8 * k1.expr)

k2 = name_expr("k2", l * (d + h/2 * k1.expr).as_explicit().cross(B2))
k3 = name_expr("k3", l * (d + h/2 * k2.expr).as_explicit().cross(B2))
p3 = name_expr("p3", p + h * d + h**2/2 * k3.expr)

k4 = name_expr("k4", l * (d + h * k3.expr).as_explicit().cross(B3))

err = name_expr("err", h**2 * (k1.expr - k2.expr - k3.expr + k4.expr).norm(1))

new_p = name_expr("new_p", p + h * d + h**2/6 * (k1.expr + k2.expr + k3.expr))
new_d = name_expr("new_d", d + h/6 * (k1.expr + 2 * (k2.expr + k3.expr) + k4.expr))
new_d_norm = name_expr("new_d", new_d.expr / new_d.expr.as_explicit().norm())

dtds = name_expr("dtds", sym.sqrt(1 + m**2 / p_abs**2))
new_time = name_expr("new_time", h * dtds.expr)

In [ ]:
D = Matrix(np.zeros((8, 8)))
D[0:3,:] = new_p.expr.subs([k3,k2,k1]).as_explicit().jacobian([p, t, d, l])
D[3,7] = h * m**2 * l / dtds.expr;
D[4:7,:] = new_d.expr.subs([k4,k3,k2,k1]).as_explicit().jacobian([p, t, d, l])
D[7,7] = 1

J = Matrix(MatrixSymbol("J", 8, 8).as_explicit())
for indices in np.ndindex(J.shape):
    if D[indices] in [0, 1]:
        J[indices] = D[indices]
J = ImmutableMatrix(J)

new_J = name_expr("new_J", J * D)

In [ ]:
# code gen

In [ ]:
cxx_printer = CXX11CodePrinter()

In [ ]:
def find_by_name(name_exprs, name):
    return next((name_expr for name_expr in name_exprs if str(name_expr[0]) == name), None)

In [ ]:
def inflate_expr(name_expr):
    name, expr = name_expr

    result = []
    references= []

    if hasattr(expr, "shape"):
        for indices in np.ndindex(expr.shape):
            result.append((name[indices], expr[indices]))
            references.append((name, expr.shape, indices))
    else:
        result.append((name, expr))
        references.append(None)

    return result, references

def inflate_exprs(name_exprs):
    result = []
    references= []
    for name_expr in name_exprs:
        res, refs = inflate_expr(name_expr)
        result.extend(res)
        references.extend(refs)
    return result, references

def deflate_exprs(name_exprs, references):
    result = []
    deflated = {}

    for name_expr, reference in zip(name_exprs, references):
        if reference is None:
            result.append(name_expr)
        else:
            _, expr = name_expr
            name, shape, indices = reference
            if name not in deflated:
                e = Matrix(np.zeros(shape))
                result.append(NamedExpr(name, e))
                deflated[name] = e
            deflated[name][*indices] = expr

    another_result = []
    for name_expr in result:
        name, expr = name_expr
        if isinstance(expr, Matrix):
            another_result.append(NamedExpr(name, ImmutableMatrix(expr)))
        else:
            another_result.append(name_expr)

    return another_result

def my_subs(expr, sub_name_exprs):
    sub_name_exprs, _ = inflate_exprs(sub_name_exprs)

    result = expr
    result = sym.expand(result)
    result = result.subs([(e,n) for n, e in sub_name_exprs])
    result = sym.simplify(result)
    return result

In [ ]:
def build_dependency_graph(name_exprs):
    graph = {}
    for name, expr in name_exprs:
        graph[name] = expr.free_symbols
    return graph

def build_influence_graph(name_exprs):
    graph = {}
    for name, expr in name_exprs:
        for s in expr.free_symbols:
            graph.setdefault(s, set()).add(name)
    return graph

In [ ]:
def order_exprs_by_input(name_exprs):
    all_expr_names = set().union(name for name, _ in name_exprs)
    all_expr_symbols = set().union(*[expr.free_symbols for _, expr in name_exprs])
    inputs = all_expr_symbols - all_expr_names

    order = {}

    order.update({i: 0 for i in inputs})

    while len(order) < len(inputs) + len(name_exprs):
        for name, expr in name_exprs:
            symbols_order = [order.get(s, None) for s in expr.free_symbols]
            if None in symbols_order:
                continue
            order[name] = max(symbols_order) + 1

    result = name_exprs
    result = sorted(result, key=lambda n_e: len(n_e[1].args))
    result = sorted(result, key=lambda n_e: len(n_e[1].free_symbols))
    result = sorted(result, key=lambda n_e: order[n_e[0]])
    return result

In [ ]:
def order_exprs_by_output(name_exprs, outputs):
    name_expr_by_name = {name_expr[0]: name_expr for name_expr in name_exprs}

    def get_inputs(output): 
        name_expr = name_expr_by_name.get(output, None)
        if name_expr is None:
            return set()
        inputs = set(name_expr[1].free_symbols)
        inputs.update(*[get_inputs(name) for name in inputs])
        return inputs

    result = []
    done = set()

    for output in outputs:
        inputs = get_inputs(output) - done
        result.extend(order_exprs_by_input([name_exprs for name_exprs in name_exprs if name_exprs[0] in inputs]))
        result.append(name_expr_by_name[output])
        done.update(inputs)
        done.add(output)

    return result

In [ ]:
def my_cse(name_exprs, inflate_deflate=True):
    sub_symbols = numbered_symbols()

    if inflate_deflate:
        name_exprs, references = inflate_exprs(name_exprs)

    names = [x[0] for x in name_exprs]
    exprs = [x[1] for x in name_exprs]

    sub_exprs, simp_exprs = sym.cse(exprs, symbols=sub_symbols)

    simp_name_exprs = list(zip(names, simp_exprs))
    if inflate_deflate:
        simp_name_exprs = deflate_exprs(simp_name_exprs, references)

    name_exprs = []
    name_exprs.extend(sub_exprs)
    name_exprs.extend(simp_name_exprs)

    return name_exprs

In [ ]:
def my_expression_print(printer, name_exprs, outputs, run_cse=True, pre_expr_hook=None, post_expr_hook=None):
    def print_assign(var, expr):
        code = printer.doprint(Assignment(var, expr))
        return code

    if run_cse:
        name_exprs = my_cse(name_exprs, inflate_deflate=True)
    name_exprs = order_exprs_by_output(name_exprs, outputs)

    lines = []

    for var, expr in name_exprs:
        if pre_expr_hook is not None:
            code = pre_expr_hook(var)
            if code is not None:
                lines.extend(code.split("\n"))

        code = printer.doprint(Assignment(var, expr))
        if var not in outputs:
            if hasattr(expr, "shape"):
                lines.append(f"T {var}[{np.prod(expr.shape)}];")
                lines.extend(code.split("\n"))
            else:
                lines.append("const auto " + code)
        else:
            if hasattr(expr, "shape"):
                lines.extend(code.split("\n"))
            else:
                lines.append("*" + code)

        if post_expr_hook is not None:
            code = post_expr_hook(var)
            if code is not None:
                lines.extend(code.split("\n"))

    return "\n".join(lines)

def my_function_print(printer, name, inputs, name_exprs, outputs, run_cse=True):
    def input_param(input):
        if isinstance(input, MatrixSymbol):
            return f"const T* {input.name}"
        return f"const T {input.name}"
    def output_param(name):
        return f"T* {name}"

    lines = []

    params = [input_param(input) for input in inputs] + [output_param(output) for output in outputs]
    head = f"template<typename T> void {name}({", ".join(params)}) {{"

    lines.append(head)

    code = my_expression_print(printer, name_exprs, outputs, run_cse=run_cse)
    lines.extend([f"  {l}" for l in code.split("\n")])

    lines.append("}")

    return "\n".join(lines)

def my_step_function_print(name_exprs, run_cse=True):
    printer = cxx_printer
    name = "rk4"
    inputs = []
    outputs = [find_by_name(name_exprs, name)[0] for name in ["p2", "p3", "err", "new_p", "new_d", "new_time", "new_J"]]

    lines = []

    head = "template <typename T, typename GetB> bool rk4(const T* p, const T* d, const T h, const T lambda, const T m, const T p_abs, GetB getB, T* new_p, T* new_d, T* new_time, T* J) {"
    lines.append(head)

    lines.append("  const auto B1 = getB(p);");

    def pre_expr_hook(var):
        if str(var) == "p2":
            return "T p2[3];"
        if str(var) == "p3":
            return "T p3[3];"
        if str(var) == "err":
            return "T err;"
        if str(var) == "new_J":
            return "T new_J[64];"
        return None
    def post_expr_hook(var):
        if str(var) == "p2":
            return "const auto B2 = getB(p2);"
        if str(var) == "p3":
            return "const auto B3 = getB(p3);"
        if str(var) == "err":
            return "if (err > 1e-4) {\n  return false;\n}"
        if str(var) == "new_time":
            return "if (J == nullptr) {\n  return true;\n}"
        if str(var) == "new_J":
            return printer.doprint(Assignment(MatrixSymbol("J", 8, 8), var))
        return None

    code = my_expression_print(
        printer,
        name_exprs,
        outputs,
        run_cse=run_cse,
        pre_expr_hook=pre_expr_hook,
        post_expr_hook=post_expr_hook,
    )
    code = code.replace("*err", "err")
    lines.extend([f"  {l}" for l in code.split("\n")])

    lines.append("  return true;")

    lines.append("}")

    return "\n".join(lines)

In [ ]:
code = my_step_function_print(
    [k1, p2, k2, k3, p3, k4, err, new_p, new_d, dtds, new_time, dk2dTL, dk3dTL, dk4dTL, dFdTL, dGdTL, new_J],
    run_cse=True,
)
print(code)

In [ ]:
# try som manual cse

In [ ]:
name_exprs = [B1, k1, p2, B2, k2, k3, p3, B3, k4, err, new_p, new_d, dk2dTL, dk3dTL, dk4dTL, dFdTL, dGdTL, J_new]
sub_name_exprs = [
    name_expr("lB1", l * B1),
    name_expr("lB2", l * B2),
    name_expr("lB3", l * B3),
]

name_exprs = [NamedExpr(name, my_subs(expr, sub_name_exprs)) for name, expr in name_exprs]

code = my_function_print(
    cxx_printer,
    "rk4",
    [p, d, h, l],
    name_exprs,
    [err.name, new_p.name, new_d.name, J_new.name],
    run_cse=False,
)
print(code)